<a href="https://colab.research.google.com/github/minh041104/Final-Project-AIT2003-1/blob/main/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

**Xem qua dữ liệu thô chưa được xử lí**

In [2]:
path="thongtinchinhphu.csv"

In [ ]:
raw_df = pd.read_csv('thongtinchinhphu.csv')
raw_df.info()

**Xóa những cột không cần thiết cho việc phân tích**

In [ ]:
raw_df.dropna(axis = 1, thresh = 30, inplace = True)
raw_df.info()

**Giữ lại những cột quan trọng phù hợp cho mục đích phân tích**

In [ ]:
selected_columns = ['post_id', 'post_text', 'time', 'comments', 'comments_full', 'shares', 'post_url', 'reactors', 'reactions', 'reaction_count', 'fetched_time']
raw_df = raw_df[selected_columns]
raw_df.info()

**Chuyển đổi kiểu dữ liệu cho phù hợp**

In [ ]:
raw_df['post_id'] = raw_df['post_id'].astype('string')
raw_df['post_text'] = raw_df['post_text'].astype('string')
raw_df['post_url'] = raw_df['post_url'].astype('string')
raw_df['reactions'] = raw_df['reactions'].astype('string')
raw_df['time'] = pd.to_datetime(raw_df['time'])
raw_df['fetched_time'] = pd.to_datetime(raw_df['fetched_time'])
raw_df.info()

**Xóa dữ liệu không cần thiết**

In [ ]:
raw_df[raw_df["reactors"].isnull()].index

In [ ]:
index_to_drop = raw_df[raw_df["reactors"].isnull()].index
raw_df.drop(index_to_drop, inplace=True)
raw_df.info()

**Xử lý dữ liệu bị lặp lại**

In [ ]:
raw_df.drop_duplicates(subset= 'post_id', inplace= True)
raw_df.info()

**Điền dữ liệu bị thiếu**

In [ ]:
import ast
# Định nghĩa hàm để thêm các key còn thiếu và đặt giá trị là 0
def fill_missing_keys(reactions_str):
    try:
        reactions_dict = ast.literal_eval(reactions_str)
    except (ValueError, SyntaxError):
        # Xử lý nếu không thể chuyển đổi thành từ điển
        reactions_dict = {}

    all_keys = ['thích', 'yêu thích', 'haha', 'wow', 'thương thương', 'buồn', 'phẫn nộ']
    return {key: reactions_dict.get(key, 0) for key in all_keys}

# Áp dụng hàm cho cột 'reactions' và điền thiếu dữ liệu
raw_df['reactions'] = raw_df['reactions'].apply(lambda x: fill_missing_keys(x))

# Hiển thị cột reactions sau khi điền dữ liệu bị thiếu
raw_df['reactions'].value_counts()

**Thêm một số cột để hỗ trợ cho việc phân tích**

In [ ]:
# Sử dụng json_normalize để tách từng key thành các cột riêng biệt
reactions_normalized = pd.json_normalize(raw_df['reactions'])

# Ghép DataFrame mới vào DataFrame cũ
raw_df = pd.concat([raw_df, reactions_normalized], axis=1)
raw_df.info()

**Đổi tên cột sang tiếng anh**

In [ ]:
raw_df.rename(columns={
'thích': 'like',
'yêu thích': 'love',
'thương thương': 'care',
'buồn': 'sad',
'phẫn nộ': 'angry'}, inplace=True)

**Thời gian lúc mới đăng bài đến khi dữ liệu được thu thập**

In [ ]:
raw_df['time_difference'] = raw_df['fetched_time'] - raw_df['time']
raw_df.info()

**Lưu dữ liệu đã được làm sạch**

In [ ]:
raw_df.to_csv('clean_df.csv', index=False)